In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from IPython.display import HTML
import sys

In [2]:
# code from coursework of ACSE1
def life(initial_state, nsteps, periodic=False):
    """
    Perform iterations of Conway’s Game of Life.
    Parameters
    ----------
    initial_state : array_like or list of lists
        Initial 2d state of grid in an array of booleans.
    nsteps : int
        Number of steps of Life to perform.
    periodic : bool
        If true, then grid is assumed periodic.
    Returns
    -------
    numpy.ndarray
         Final state of grid in array of booleans
    """
    if periodic:
        # ensure intial_state is array
        current_state = np.array(initial_state)
        while nsteps > 0:
            current = current_state.astype(int)
            # find neighbors for each cell
            sum_neighbors = sum(np.roll(np.roll(current, i, axis=0), j, axis=1) \
                               for i in (1, 0, -1) for j in (1, 0, -1))-current
            # find survival items
            current_state = np.logical_or(sum_neighbors == 3, np.logical_and( \
                              sum_neighbors == 2, current_state))
            nsteps -= 1
    else:
        # ensure initial_state is array
        current_state = np.array(initial_state)
        while nsteps > 0:
            current = current_state.astype(int)
            # pad the array
            current = np.pad(current, ((1, 1), (1, 1)), 'constant', constant_values=(0, 0))
            # find neighbors for each cell
            sum_neighbors = sum(np.roll(np.roll(current, i, axis=0), j, axis=1) \
                            for i in (1, 0, -1) for j in (1, 0, -1))-current
            # find survival items
            sum_neighbors = sum_neighbors[1:-1, 1:-1]
            current_state = np.logical_or(sum_neighbors == 3, \
                           np.logical_and(sum_neighbors == 2, current_state))
            nsteps -= 1
    return current_state

 In mpi code I divide processes into 2D grid. In the size.txt, it stores the size of this 2D grid(number of rows and colums), number of steps of life of game to game and boundary condition(periodic or not).

In [3]:
#read the size.txt, get the necessary information
f = open("size.txt")
line = f.readline()
while line:  
    row, col, itor,periodical = list(map(int, line.split()))
    line = f.readline()
f.close()

compare with the python code, ensure results are correct

In [4]:
file = str(0) + "_"
data = []
for i in range(row*col):
    f = open(file+str(i)+".txt")
    line = f.readline()
    tmp = []
    while line:  
        tmp.append(list(map(float, line.split())))
        line = f.readline()
    data.append(np.array(tmp))
    f.close()
initial = []
for i in range(row):
    initial.append(np.concatenate(data[i*col: (i+1)*col], axis=1))
initial = np.concatenate(initial, axis=0)

In [9]:
initial[0]

array([0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1.])

In [5]:
file = str(itor) + "_"
data = []
for i in range(row*col):
    f = open(file+str(i)+".txt")
    line = f.readline()
    tmp = []
    while line:  
        tmp.append(list(map(float, line.split())))
        line = f.readline()
    data.append(np.array(tmp))
    f.close()
final = []
for i in range(row):
    final.append(np.concatenate(data[i*col: (i+1)*col], axis=1))
final = np.concatenate(final, axis=0)

In [6]:
ccc = life(initial,itor,periodical)
assert((ccc==final).all())

make the movie

In [7]:
# code from coursework of ACSE2
def create_video():    
    fig = plt.figure(figsize=(10,8))
    imgs = []
    for j in range(itor+1):
        file = str(j) + "_"
        data = []
        for i in range(row*col):
            f = open(file+str(i)+".txt")
            line = f.readline()
            tmp = []
            while line:  
                tmp.append(list(map(float, line.split())))
                line = f.readline()
            data.append(np.array(tmp))
            f.close()
        ccc = []
        for i in range(row):
            ccc.append(np.concatenate(data[i*col: (i+1)*col], axis=1))
        ccc = np.concatenate(ccc, axis=0)
        img = plt.imshow(ccc, cmap='binary')
        imgs.append([img])
    ani = anim.ArtistAnimation(fig, imgs, interval=200, blit=True)
    plt.close(fig)  # prevent final frame plot from showing up inline below

    return ani


In [8]:
ani = create_video()
print('Preparing HTML (takes a little while...)')
HTML(ani.to_jshtml())

Preparing HTML (takes a little while...)
